In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
from config import g_key
gmaps.configure(api_key=g_key)

In [4]:
# Import the WeatherPy_vacation.csv
city_data_df = pd.read_csv("data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hours),Snow inches (last 3 hours)
0,0,Punta Arenas,CL,2020-02-23 21:17:15,-53.15,-70.92,55.40,62,75,32.21,broken clouds,0.0,0.0
1,1,Hermanus,ZA,2020-02-23 21:19:29,-34.42,19.23,65.71,75,0,26.80,clear sky,0.0,0.0
2,2,Castro,BR,2020-02-23 21:19:29,-24.79,-50.01,67.93,78,90,2.66,overcast clouds,0.0,0.0
3,3,Albany,US,2020-02-23 21:16:59,42.60,-73.97,51.80,29,75,5.82,broken clouds,0.0,0.0
4,4,Okhotsk,RU,2020-02-23 21:19:29,59.38,143.30,-14.96,77,16,7.90,few clouds,0.0,0.0


In [5]:
# Prompt the customer for the minimum temperature preference
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65


In [6]:
# Prompt the customer for the maximum temperature preference
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the maximum temperature you would like for your trip? 70


In [7]:
# Prompt the customer to answer if he or she would like it to be raining or not
rain_check = input("Do you want it to be raining? (yes/no) ")

# When the user enters anything else beside yes and no
while not(rain_check == "yes" or rain_check == "no"):
    rain_check = input("Do you want it to be raining? (yes/no) ")
    
# Check rain based on user input
rain_condition = (city_data_df["Rain inches (last 3 hours)"] > 0) if rain_check == "yes" else (city_data_df["Rain inches (last 3 hours)"] == 0)

Do you want it to be raining? (yes/no) no


In [8]:
# Prompt the customer to answer if he or she would like it to be snowing or not
snow_check = input("Do you want it to be snowing? (yes/no) ")

# When the user enters anything else beside yes and no
while not(snow_check == "yes" or snow_check == "no"):
    snow_check = input("Do you want it to be snowing? (yes/no) ")

# Conditional for checking snow based on user input
snow_condition = (city_data_df["Snow inches (last 3 hours)"] > 0) if snow_check == "yes" else (city_data_df["Snow inches (last 3 hours)"] == 0)

Do you want it to be snowing? (yes/no) no


In [9]:
# Filter the DataFrame based on user preferences preferences
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp) &
                                      rain_condition & snow_condition] 
preferred_cities_df.head()
if len(preferred_cities_df) == 0:
    print("No cities match your criteria")

In [10]:
# Add the cities to a marker layer map with a pop-up marker for each city that includes:

In [11]:
# Create hotel dataframe with name, city, country, current weather description, max temperture
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hermanus,ZA,65.71,clear sky,-34.42,19.23,
2,Castro,BR,67.93,overcast clouds,-24.79,-50.01,
5,Buraydah,SA,68.00,scattered clouds,26.33,43.98,
7,Airai,TL,66.69,clear sky,-8.93,125.41,
17,Cape Town,ZA,66.99,clear sky,-33.93,18.42,
38,Jobner,IN,66.20,haze,26.97,75.38,
43,Busselton,AU,68.00,broken clouds,-33.65,115.33,
53,Sao Filipe,CV,68.41,few clouds,14.90,-24.50,
72,Coihaique,CL,66.20,broken clouds,-45.58,-72.07,
82,Bredasdorp,ZA,66.20,clear sky,-34.53,20.04,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hermanus,ZA,65.71,clear sky,-34.42,19.23,Misty Waves Boutique Hotel
2,Castro,BR,67.93,overcast clouds,-24.79,-50.01,CHACARA BAILLY
5,Buraydah,SA,68.00,scattered clouds,26.33,43.98,Mövenpick Hotel Qassim
7,Airai,TL,66.69,clear sky,-8.93,125.41,Juvinal Martin
17,Cape Town,ZA,66.99,clear sky,-33.93,18.42,Southern Sun Waterfront Cape Town
38,Jobner,IN,66.20,haze,26.97,75.38,Srk Jobner Hostel
43,Busselton,AU,68.00,broken clouds,-33.65,115.33,Observatory Guest House
53,Sao Filipe,CV,68.41,few clouds,14.90,-24.50,Tortuga B&B
72,Coihaique,CL,66.20,broken clouds,-45.58,-72.07,Diego De Almagro Coyhaique
82,Bredasdorp,ZA,66.20,clear sky,-34.53,20.04,Bredasdorp Country Manor


In [14]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Curent Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
# Add pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Save and upload the new DataFrame as WeatherPy_vacation.csv
output_data_file = "data/WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file, index_label="City_ID")